<a href="https://colab.research.google.com/github/Arsalan9002/Apache-Beam/blob/master/ApacheBeam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!{'pip install apache-beam[gcp]'}

     |████████████████████████████████| 8.3MB 2.8MB/s 
     |████████████████████████████████| 1.4MB 46.1MB/s 
     |████████████████████████████████| 829kB 46.8MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 153kB 46.5MB/s 
     |████████████████████████████████| 63.8MB 61kB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 61kB 7.8MB/s 
     |████████████████████████████████| 122kB 47.2MB/s 
     |████████████████████████████████| 215kB 33.1MB/s 
     |████████████████████████████████| 184kB 48.4MB/s 
     |████████████████████████████████| 235kB 48.2MB/s 
     |████████████████████████████████| 92kB 10.0MB/s 
     |████████████████████████████████| 92kB 11.4MB/s 
     |████████████████████████████████| 174kB 52.1MB/s 
     |████████████████████████████████| 440kB 43.7MB/s 
     |████████████████████████████████| 153kB 50.1MB/s 
     |████████████████████████████████| 112kB 46.1MB/s 
  

In [4]:
!{'mkdir -p data'}

In [5]:
!ls

data  sample_data


In [6]:
#to import local files into google colab
from google.colab import files
uploaded = files.upload()

Saving data.txt to data.txt
Saving demo_code.txt to demo_code.txt
Saving dept_data.txt to dept_data.txt


In [ ]:
!cat dept_data.txt

In [8]:
import apache_beam as beam
p2 = beam.Pipeline()

lines = (
      p2
      | beam.Create(['Arsalan','Mehmood','Systems Limited','Karachi'])
      | beam.io.WriteToText('data/outCreate1')
)

p2.run()

!{('head -n 20 data/outCreate1-00000-of-00001')}

AttributeError: ignored

In [ ]:
import apache_beam as beam
p2 = beam.Pipeline()

lines = (
      p2
      | beam.Create({'row1':[1,2,3,4,5],
                     'row2':[6,7,8,9]
                     })
      | beam.io.WriteToText('data/outCreate1')
)

p2.run()

!{('head -n 20 data/outCreate1-00000-of-00001')}

In [ ]:
#create a pipeline to show the attendance count of each employee of finance department
import apache_beam as beam

p1 = beam.Pipeline()

attendance_count = (
    p1
    | 'Reading dept data' >> beam.io.ReadFromText('dept_data.txt')
    | 'Splitting records' >> beam.Map(lambda x: x.split(','))
    | 'Filtering Accounts dept' >> beam.Filter(lambda x: x[3]=='Accounts')
    | 'Creating Key value pairs' >> beam.Map(lambda x: (x[1],1))
    | 'Grouping & counting Keys' >> beam.CombinePerKey(sum)
    | 'Writing to file' >> beam.io.WriteToText('data/output_new_final')
)

p1.run()

!{('head -n 20 data/output_new_final-00000-of-00001')}

In [ ]:
#to view the output file
!cat data/output_new_final-00000-of-00001

In [1]:
#branching pipelines
import apache_beam as beam

with beam.Pipeline() as p:
  input_collection = (
      p
      | 'reading department data' >> beam.io.ReadFromText('dept_data.txt')
      | 'split rows' >> beam.Map(lambda x: x.split(','))
  )

  accounts_attendance = (
      input_collection
      | 'filter for accounts' >> beam.Filter(lambda x: x[3]=='Accounts')
      | 'pair each accounts employee with 1' >> beam.Map(lambda x: ("Accounts, " + x[1],1))
      | 'Grouping and Sum1' >> beam.CombinePerKey(sum)
  )

  hr_attendance = (
      input_collection
      | 'filter for HR' >> beam.Filter(lambda x: x[3]=='HR')
      | 'pair each HR employee with 1' >> beam.Map(lambda x: ("HR, "+x[1],1))
      | 'Grouping and Sum' >> beam.CombinePerKey(sum)
  )

  output = (
      (accounts_attendance, hr_attendance)
      | 'Merging two PCollections' >> beam.Flatten()
      | 'Writing output' >> beam.io.WriteToText("data/attendance_results")
  )


!{('head -n 20 data/attendance_results-00000-of-00001')}

('Accounts, Marco', 31)
('Accounts, Rebekah', 31)
('Accounts, Itoe', 31)
('Accounts, Edouard', 31)
('Accounts, Kyle', 62)
('Accounts, Kumiko', 31)
('Accounts, Gaston', 31)
('Accounts, Ayumi', 30)
('HR, Beryl', 62)
('HR, Olga', 31)
('HR, Leslie', 31)
('HR, Mindy', 31)
('HR, Vicky', 31)
('HR, Richard', 31)
('HR, Kirk', 31)
('HR, Kaori', 31)
('HR, Oscar', 31)


In [ ]:
#word count exmaple
import apache_beam as beam
import re

#https://regex101.com/
def regex(element):
  x=element
  y = re.sub(r"[^a-zA-Z0-9]+", '', x)
  return y

p = beam.Pipeline()

word_count = (
    p
    | 'Read from file' >> beam.io.ReadFromText('data.txt')
    | 'Split words' >> beam.FlatMap(lambda x: x.split(" "))
    | 'split tabs' >> beam.FlatMap(lambda x: x.split("\t"))
    | 'trim' >> beam.Map(lambda x: x.strip()) #for safer side
    | 'remove empty elements' >> beam.Filter(lambda x: not re.match(r'^\s*$', x))
    | 'removing special characters' >> beam.Map(regex)
    | 'emitting 1 for every unique word' >> beam.Map(lambda x: (x,1))
    | 'reducing & summing' >> beam.CombinePerKey(sum)
    | 'Write output to file' >> beam.io.WriteToText("data/output_wordcount")

)

p.run()

!head -n 20 data/output_wordcount*

In [5]:
import apache_beam as beam

class SplitRow(beam.DoFn):
  #override process function
  def process(self,element):
    #returning list since this can return one or more outputs
    return [element.split(',')]

p = beam.Pipeline()

attendance_count = (
    p
    | beam.io.ReadFromText('dept_data.txt')
    | beam.ParDo(SplitRow())
    | beam.io.WriteToText("data/pardo_output")
)

p.run()
 
!{('head -n 20 data/pardo_output-00000-of-00001')}

['149633CM', 'Marco', '10', 'Accounts', '1-01-2019']
['212539MU', 'Rebekah', '10', 'Accounts', '1-01-2019']
['231555ZZ', 'Itoe', '10', 'Accounts', '1-01-2019']
['503996WI', 'Edouard', '10', 'Accounts', '1-01-2019']
['704275DC', 'Kyle', '10', 'Accounts', '1-01-2019']
['957149WC', 'Kyle', '10', 'Accounts', '1-01-2019']
['241316NX', 'Kumiko', '10', 'Accounts', '1-01-2019']
['796656IE', 'Gaston', '10', 'Accounts', '1-01-2019']
['331593PS', 'Beryl', '20', 'HR', '1-01-2019']
['560447WH', 'Olga', '20', 'HR', '1-01-2019']
['222997TJ', 'Leslie', '20', 'HR', '1-01-2019']
['171752SY', 'Mindy', '20', 'HR', '1-01-2019']
['153636AS', 'Vicky', '20', 'HR', '1-01-2019']
['745411HT', 'Richard', '20', 'HR', '1-01-2019']
['298464HN', 'Kirk', '20', 'HR', '1-01-2019']
['783950BW', 'Kaori', '20', 'HR', '1-01-2019']
['892691AR', 'Beryl', '20', 'HR', '1-01-2019']
['245668UZ', 'Oscar', '20', 'HR', '1-01-2019']
['231206QD', 'Kumiko', '30', 'Finance', '1-01-2019']
['357919KT', 'Wendy', '30', 'Finance', '1-01-2019